<a href="https://colab.research.google.com/github/Lior-Baruch/SFT_llama_qlora/blob/main/DPO_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install all dependencies
!pip install -q -U peft transformers datasets bitsandbytes trl accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
# all imports
import warnings
warnings.filterwarnings("ignore")
from huggingface_hub import notebook_login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer


In [3]:
from huggingface_hub import notebook_login

# log in to the Hugging Face hub (required for private datasets/models)
notebook_login()

LoRA adapter

In [ ]:
# Define LoRA ("low-rank attention") config
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

load our saved model from the hub

In [4]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = f"LBK95/llama-7b-qlora-ultrachat"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model_loaded = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)



tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

test our base model vs fine-tuned model

In [14]:
do_test = False

if do_test:
    # test our base model vs fine-tuned model
    text = "### USER: How to plan a picnic?### Assistant:"
    inputs = tokenizer(text, return_tensors="pt").to(0)

    # with adaptor (fine-tuned model)
    model_loaded.enable_adapters()
    outputs = model_loaded.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

    print("After attaching Lora adapters:")
    print(tokenizer.decode(outputs[0], skip_special_tokens=False))
    print()

    # without adaptor (base model)
    model_loaded.disable_adapters()
    outputs = model_loaded.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

    print("Before attaching Lora adapters:")
    print(tokenizer.decode(outputs[0], skip_special_tokens=False))

    # enable adapters
    model_loaded.enable_adapters()

# DPO

Load human prefrence dataset

In [6]:
dataset = load_dataset(
    "lvwerra/stack-exchange-paired",
    split="train[:1%]",
    data_dir="data/rl"
)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
def return_prompt_and_responses(samples):
    return {
        "prompt": [
            "Question: " + question + "\n\nAnswer: "
            for question in samples["question"]
        ],
        "chosen": samples["response_j"],   # rated better than k
        "rejected": samples["response_k"], # rated worse than j
    }

original_columns = dataset.column_names
print(original_columns)

dataset_pref = dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)
print(dataset_pref.column_names)
print(dataset_pref.shape)

['qid', 'question', 'date', 'metadata', 'response_j', 'response_k']


Map:   0%|          | 0/74359 [00:00<?, ? examples/s]

['prompt', 'chosen', 'rejected']
(74359, 3)


simple EDA

In [15]:
# very simple eda on the dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

perform_EDA = False

if perform_EDA:
    df = pd.DataFrame(dataset_pref[:1000])
    df.head()

    df["prompt_length"] = df["prompt"].apply(lambda x: len(x.split()))
    df["chosen_length"] = df["chosen"].apply(lambda x: len(x.split()))
    df["rejected_length"] = df["rejected"].apply(lambda x: len(x.split()))

    df.describe()

    sns.histplot(df["prompt_length"], bins=20)
    plt.show()

    sns.histplot(df["chosen_length"], bins=20)
    plt.show()

    sns.histplot(df["rejected_length"], bins=20)
    plt.show()

training args

In [9]:
from transformers import TrainingArguments

# Define training arguments
YOUR_HF_USERNAME = "LBK95" # Set my HF username

output_dir = f"{YOUR_HF_USERNAME}/llama-7b-qlora-ultrachat-DPO" # Set output directory for fine-tuned model
per_device_train_batch_size = 4  # Set batch size per device for training
gradient_accumulation_steps = 4  # Set the number of gradient accumulation steps
optim = "paged_adamw_32bit" # Set optimizer
save_steps = 10 # Set the number of steps before saving
logging_steps = 10 # Set the number of steps before logging
learning_rate = 2e-4 # Set learning rate
max_grad_norm = 0.3 # Set max gradient norm
max_steps = 200 # Set max steps
warmup_ratio = 0.03 # Set warmup ratio for learning rate scheduler
lr_scheduler_type = "constant" # Set learning rate scheduler type


# Create TrainingArguments object with all of our defined arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)

load model ref

In [10]:
model_ref = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
dataset_pref_1000 = dataset_pref.select(range(1000))

create DPO trainer and train

In [12]:
from trl import DPOTrainer

dpo_trainer = DPOTrainer(
    model_loaded,          # base model from SFT pipeline
    model_ref,             # typically a copy of the SFT trained base model
    beta=0.1,              # temperature hyperparameter of DPO
    train_dataset=dataset_pref_1000, # dataset prepared above
    tokenizer=tokenizer,   # tokenizer
    args=training_arguments,    # training arguments e.g. batch size, lr, etc.
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
dpo_trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
